In [ ]:
! pip install pymysql

Install Python 3.9.13

In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.feature_selection as fs
import pickle
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as preprocessing
from torch.utils.data import Dataset, DataLoader
from torch.nn.functional import normalize
from sklearn.svm import SVR, NuSVR
from sklearn.pipeline import make_pipeline

# Grab Dataset

In [27]:
ysi = pd.read_csv("../datasets/ysi_dataset.csv")
weather = pd.read_csv("../datasets/weather_dataset.csv")
# convert to date time and round ysi to nearest 15 mins
ysi['datetime'] = pd.to_datetime(ysi['datetime']).dt.round('15min')
weather['datetime'] = pd.to_datetime(weather['datetime'])

In [28]:
# combine both DF's on datetime
combined = pd.merge(ysi, weather, on='datetime')
combined.reset_index(drop=True, inplace=True)
combined = combined.drop(columns=['experimentid_y'])
combined.columns

Index(['datetime', 'experimentid_x', 'ph', 'temperature_oc',
       'par_umol_photons_m2_s', 'dissolved_oxygen_mg_l', 'airtemp_oc',
       'global_light_energy_w_m2', 'humid_rh', 'wdspd_m_s'],
      dtype='object')

# Data Visualization

In [29]:
# combined = combined.drop(combined[combined['ph'] < 5].index)
# combined = combined.drop(combined[combined['ph'] > 9].index)
combined = combined.drop(combined[combined['dissolved_oxygen_mg_l'] < 2].index)
combined = combined.drop(combined[combined['dissolved_oxygen_mg_l'] > 20].index)
combined = combined.drop(combined[combined['dissolved_oxygen_mg_l'] < 3/100 * combined['global_light_energy_w_m2']].index)
combined = combined.drop(combined[combined['dissolved_oxygen_mg_l'] >( 3/100 * combined['global_light_energy_w_m2']) + 15].index)

In [30]:
combined.to_csv("../datasets/do_combined.csv", index=False)

In [ ]:
feat = combined.sample(3000)
plt.plot(feat['temperature_oc'],feat['dissolved_oxygen_mg_l'], 'bo')
plt.ylabel("ph")
plt.xlabel("global_light_energy_w_m2")
plt.show()

In [10]:
combined.to_csv("../datasets/do_combined.csv", index=False)

# Feature Selection

In [ ]:
X = feat[['ph', 'temperature_oc', 'global_light_energy_w_m2', 'humid_rh']].values
y = feat['dissolved_oxygen_mg_l'].values
regressor = RandomForestRegressor(n_estimators=50)
regressor.fit(X, y)


print(regressor.feature_importances_)

model = SelectFromModel(regressor, prefit=True)
X_new = model.transform(X)

features selected: ph, temperature_oc, global_light_energy_w_m2, humid_rh
not considered: wdspd_m_s (low correlation), par_umol_photons_m2_s (duplicated),

# Training

In [41]:
# store all rows where temp is greater than 25 in splitted
splitted = pd.read_csv("../../../../../../Downloads/do_combined_updated_new.csv")
print(splitted.columns)
test = splitted.copy(True)
X_train,X_test,y_train,y_test = train_test_split(test[['global_light_energy_w_m2', 'ph', 'wdspd_m_s_airtemp_oc_ph', 'humid_rh']], 
        test['dissolved_oxygen_mg_l'], test_size=0.15)

Index(['datetime', 'experimentid_x', 'ph', 'temperature_oc',
       'par_umol_photons_m2_s', 'dissolved_oxygen_mg_l', 'airtemp_oc',
       'global_light_energy_w_m2', 'humid_rh', 'wdspd_m_s', 'wdspd_m_s_ph',
       'wdspd_m_s_airtemp_oc_ph', 'airtemp_oc_ph',
       'log_ph_global_light_energy_w_m2',
       'log_temperature_oc_global_light_energy_w_m2'],
      dtype='object')


In [32]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math

def stats(y_pred_all, y_test_all):
    # Calculate the R2 score
    r2 = r2_score(y_test_all, y_pred_all)

    print(f"R2 Score: {r2:.4f}")
    # Calculate MAE
    mae = mean_absolute_error(y_test_all, y_pred_all)

    # Calculate RMSE
    mse = mean_squared_error(y_test_all, y_pred_all, squared=False)

    print("Mean Absolute Error (MAE):", mae)
    print("Root Mean Squared Error (RMSE):", math.sqrt(mse))

In [33]:
print("------------ Linear Regression ------------")
from sklearn.linear_model import LinearRegression
linreg = LinearRegression().fit(X_train, y_train)
y_pred = linreg.predict(X_test)
stats(y_pred, y_test)

------------ Linear Regression ------------
R2 Score: 0.6737
Mean Absolute Error (MAE): 1.9003851362098825
Root Mean Squared Error (RMSE): 1.5464444546748186


In [42]:
print("------------ Exponential SVR Results ------------")
clf_rbf_nusvm = make_pipeline(preprocessing.SplineTransformer(), NuSVR(kernel='rbf', shrinking=True, C=1.5))
clf_rbf_nusvm.fit(X_train, y_train)
y_pred = clf_rbf_nusvm.predict(X_test)
stats(y_pred, y_test)

print("------------ Exponential SVR Results ------------")
clf_rbf_svm = make_pipeline(preprocessing.SplineTransformer(), SVR(kernel='rbf', shrinking=True, C=1.5))
clf_rbf_svm.fit(X_train[:len(X_train)//2], y_train[:len(X_train)//2])
clf_rbf_svm.fit(X_train[len(X_train)//2:], y_train[len(X_train)//2:])
y_pred = clf_rbf_svm.predict(X_test)
stats(y_pred, y_test)

print("------------ Polynomial SVR Results ------------")
clf_poly_nusvm = make_pipeline(preprocessing.SplineTransformer(), NuSVR(kernel='poly', shrinking=False, C=2.5))
clf_poly_nusvm.fit(X_train, y_train)
y_pred = clf_poly_nusvm.predict(X_test)
stats(y_pred, y_test)

------------ Exponential SVR Results ------------
R2 Score: 0.7777
Mean Absolute Error (MAE): 1.382567517011247
Root Mean Squared Error (RMSE): 1.3940193766137419
------------ Exponential SVR Results ------------
R2 Score: 0.7700
Mean Absolute Error (MAE): 1.3942909200906513
Root Mean Squared Error (RMSE): 1.4059255679507698
------------ Polynomial SVR Results ------------
R2 Score: 0.7828
Mean Absolute Error (MAE): 1.3479054239422932
Root Mean Squared Error (RMSE): 1.385961798084692


In [44]:
print("------------ Bagging Results ------------")
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
base_estimator = DecisionTreeRegressor()
bag_pipe = make_pipeline(preprocessing.SplineTransformer(), BaggingRegressor(base_estimator=base_estimator, n_estimators=10, n_jobs=5))
bag_pipe.fit(X_train, y_train)
y_pred = bag_pipe.predict(X_test)
stats(y_pred, y_test)


print("------------ Stacking Results ------------")
from sklearn.ensemble import StackingRegressor
model = StackingRegressor(estimators=[
    ('svr', NuSVR(kernel='poly', shrinking=False, C=2.5)), 
    ('rf', RandomForestRegressor(n_estimators=10,random_state=42, n_jobs=8)),
    ('bag', BaggingRegressor(n_jobs=5)),
    ('bst', GradientBoostingRegressor(learning_rate=0.1, loss='huber', max_depth=6, criterion='squared_error')),
    ('n1', MLPRegressor(hidden_layer_sizes=(2,3), activation='relu')),
    ('n2', MLPRegressor(hidden_layer_sizes=(3,2), activation='tanh'))
])
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
stats(y_pred, y_test)


------------ Bagging Results ------------


R2 Score: 0.8656
Mean Absolute Error (MAE): 0.9812385261797028
Root Mean Squared Error (RMSE): 1.2292702286440744
------------ Stacking Results ------------


/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multi

R2 Score: 0.8706
Mean Absolute Error (MAE): 0.9715637935788428
Root Mean Squared Error (RMSE): 1.217751364980006


In [43]:
print("------------ Random Forest Results ------------")
rf_regressor = make_pipeline(preprocessing.SplineTransformer(), RandomForestRegressor(n_estimators=24, max_depth=20))
rf_regressor.fit(X_train, y_train)
y_pred = rf_regressor.predict(X_test)
stats(y_pred, y_test)

print("------------ Poly Regression Results ------------")
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

poly_features = PolynomialFeatures(degree=5, include_bias=False)
X_train_poly = poly_features.fit_transform(X_train)
X_test_poly = poly_features.transform(X_test)
# Initialize and fit the linear regression model
poly_reg = LinearRegression()
poly_reg.fit(X_train_poly, y_train)
# Predict the target variable for training and test sets
y_pred = poly_reg.predict(X_test_poly)
stats(y_pred, y_test)

print("------------ Decision Tree Results ------------")
from sklearn.tree import DecisionTreeRegressor
d_tree = DecisionTreeRegressor(max_depth=24)
d_tree.fit(X_train, y_train)
y_pred = d_tree.predict(X_test)
stats(y_pred, y_test)

print("------------ XG Boost Results ------------")
from sklearn.ensemble import GradientBoostingRegressor
xg_boost = GradientBoostingRegressor(learning_rate=0.1, loss='huber', max_depth=6, criterion='squared_error')
xg_boost.fit(X_train, y_train)
y_pred = xg_boost.predict(X_test)
stats(y_pred, y_test)

------------ Random Forest Results ------------
R2 Score: 0.8662
Mean Absolute Error (MAE): 0.9578322337011381
Root Mean Squared Error (RMSE): 1.227838800319483
------------ Poly Regression Results ------------
R2 Score: 0.7636
Mean Absolute Error (MAE): 1.3946363812143787
Root Mean Squared Error (RMSE): 1.4156985214599742
------------ Decision Tree Results ------------
R2 Score: 0.7815
Mean Absolute Error (MAE): 1.1219497987830105
Root Mean Squared Error (RMSE): 1.3880192102364317
------------ XG Boost Results ------------
R2 Score: 0.8436
Mean Absolute Error (MAE): 1.1239813975564932
Root Mean Squared Error (RMSE): 1.2767018004050568


In [20]:
model = nn.Sequential(
    nn.Linear(3, 16),
    nn.CELU(),
    nn.Linear(16, 16),
    nn.CELU(),
    nn.Linear(16, 4),
    nn.CELU(),
    nn.Linear(4, 1)
)

In [21]:
class MyDataset(Dataset):
    def __init__(self, dataframe):
        self.numeric_columns = ['log_temperature_oc_global_light_energy_w_m2', 'log_ph_global_light_energy_w_m2', 'wdspd_m_s_airtemp_oc_ph']
        self.transformer = preprocessing.QuantileTransformer()
        transformed_data = self.transformer.fit_transform(dataframe[self.numeric_columns].values)
        dataframe[self.numeric_columns] = transformed_data
        self.data = dataframe.dropna(subset=['dissolved_oxygen_mg_l']).reset_index(drop=True)

    def __len__(self):
        return len(self.data)
    
    def getInverseTransform(self, inputs):
        return self.quantile_transformer.inverse_transform(inputs)

    def __getitem__(self, idx):
        inputs = self.data[self.numeric_columns].iloc[idx].values.astype(np.float32)
        label = self.data['dissolved_oxygen_mg_l'].iloc[idx].astype(np.float32)
        inputs = torch.tensor(inputs)
        label = torch.tensor(label)
        return inputs, label


In [23]:
loss_fn = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.001)
num_epochs = 10

splitted = pd.read_csv("../datasets/Philly/new_features.csv")
test = splitted.copy(True)
trainset, testset = train_test_split(test, test_size=0.1)

data_loader = DataLoader(MyDataset(trainset), shuffle=True)

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    model.train()
    for inputs, labels in data_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        # Reshape labels to match the shape of outputs
        labels = labels.view(outputs.shape)

        loss = loss_fn(outputs , labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    # Calculate average loss for the epoch
    epoch_loss = running_loss / len(data_loader)
    # Print the loss for each epoch
    print(f"Epoch {epoch+1}, Training Loss: {epoch_loss:.4f}")
# Training complete

Epoch 1, Training Loss: 5.4914
Epoch 2, Training Loss: 2.5774
Epoch 3, Training Loss: 2.3107
Epoch 4, Training Loss: 2.0774
Epoch 5, Training Loss: 1.8840
Epoch 6, Training Loss: 1.7336
Epoch 7, Training Loss: 1.6207
Epoch 8, Training Loss: 1.5535
Epoch 9, Training Loss: 1.5177
Epoch 10, Training Loss: 1.5021


In [ ]:
# save weights
torch.save(model.state_dict(), './weights/weights2.sav')
model.load_state_dict(torch.load('./weights/weights2.sav'))

# weights 1 : loss = 30.98
# weights 2 : loss = 1.1218

# Evaluation

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math

# Set the model to evaluation mode
model.eval()

# Initialize lists to store true and predicted values
y_test_all = []
y_pred_all = []

newset = pd.merge(ysi, weather, on='datetime').sample(1000)

dataset = MyDataset(testset)
data_loader = DataLoader(dataset)

# Disable gradient calculation
with torch.no_grad():
    for inputs, label in data_loader:

        # Forward pass to get predictions
        y_pred = model(inputs)

        # Convert the predictions and targets to numpy arrays
        y_pred_np = y_pred.numpy()
        label_np = label.numpy()
        #print(dataset.quantile_transformer.inverse_transform(inputs.numpy()))
        #print(f"actual: { y_pred.item():3f} predicted: {label.item():3f}")

        # Append batch results to the overall lists
        y_test_all.append(label_np)
        y_pred_all.append(y_pred_np)

# Concatenate the lists into a single array
y_test_all = np.concatenate(y_test_all)
y_pred_all = np.concatenate(y_pred_all)

# Calculate the R2 score
r2 = r2_score(y_test_all, y_pred_all)

print(f"R2 Score: {r2:.4f}")
# Calculate MAE
mae = mean_absolute_error(y_test_all, y_pred_all)

# Calculate RMSE
mse = mean_squared_error(y_test_all, y_pred_all, squared=False)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", math.sqrt(mse))

# Pre - Trained Models

In [ ]:
play = nn.Sequential(
    nn.Linear(4, 8),
    nn.LeakyReLU(),
    nn.Linear(8, 8),
    nn.LeakyReLU(),
    nn.Linear(8, 1)
)

play.load_state_dict(torch.load('weights.sav'))

# Save Weights

In [ ]:
model

In [ ]:
import tarfile
import json
import io

# Save the trained model
model_filename = './weights/do_rf.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(model, file)

# Load the saved model
with open(model_filename, 'rb') as file:
    loaded_model = pickle.load(file)

# Specify the name of the pickle file and the .mar archive file
mar_filename = 'model_archive.mar'

# Create a new tar archive
with tarfile.open(mar_filename, 'w') as archive:
    # Add the pickle file to the archive
    archive.add(model_filename)

# Create a manifest file with the required metadata
manifest = {'model-file': model_filename, 'model-name': 'RandomForestRegressor'}

# Add the manifest file to the archive
with tarfile.open(mar_filename, 'a') as archive:
    manifest_string = json.dumps(manifest)
    manifest_bytes = manifest_string.encode('utf-8')
    manifest_file = tarfile.TarInfo('MANIFEST')
    manifest_file.size = len(manifest_bytes)
    archive.addfile(manifest_file, io.BytesIO(manifest_bytes))


# Check performance on DB values

In [35]:
import pymysql

# Connect to the database
connection = pymysql.connect(
    host='localhost', 
    user='root', 
    password='N@wid2003', 
    db='dma_iot_morefish_spark_farms_v3'
)

# Create a cursor object
cursor = connection.cursor()

# Get the latest data from the database and store it in a pandas dataframe
query = "SELECT dvd_ph, dvd_temp, dvd_updated_at, dvd_do FROM device_devicedata WHERE dvd_ph > 0 AND dvd_temp > 0 AND dvd_dev_id = 2 ORDER BY dvd_updated_at ASC"
df = pd.read_sql(query, connection)

# Close the connection
connection.close()
len(df)

/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


778

In [36]:
df.rename(columns={'dvd_ph': 'ph', 'dvd_temp': 'temperature', 'dvd_updated_at': 'datetime', 'dvd_do': 'do_linreg'}, inplace=True)
df['rounded_datetime'] = pd.to_datetime(df['datetime']).dt.round('H')
# spark_weather = pd.read_csv("../../../../../../Downloads/sparkfarmsweather.csv")
spark_weather = pd.read_csv("../datasets/sparkfarmsweather.csv")
spark_weather['time'] = pd.to_datetime(spark_weather['time'])
spark_weather.rename(columns={'time': 'datetime'}, inplace=True)
# Merge the weather data with the main DataFrame based on 'rounded_datetime' column
df = df.merge(spark_weather, left_on='rounded_datetime', right_on='datetime', how='left')
df = df.drop(columns=['datetime_y'])
df.rename(columns={'datetime_x': 'datetime', 'diffuse_radiation (W/m²)': 'light', 'relativehumidity_2m (%)': 'humid', 'temperature_2m (°C)': 'airtemp' }, inplace=True)
# drop all NaN values inside diffuse radiation
df = df.dropna(subset=['light'])

In [27]:
df.columns

Index(['ph', 'temperature', 'datetime', 'do_linreg', 'rounded_datetime',
       'airtemp', 'humid', 'precipitation (mm)', 'pressure_msl (hPa)',
       'direct_radiation (W/m²)', 'light'],
      dtype='object')

In [37]:
x_train = df[['ph', 'temperature', 'light', 'humid']]
y_pred = rf_regressor.predict(x_train)
df['RF'] = y_pred

# test the neural net model and store its result in the pandas df
'''
y_test_all = []
y_pred_all = []

dataset = MyDataset(x_train)
data_loader = DataLoader(dataset)

# Disable gradient calculation
with torch.no_grad():
    for inputs, label in data_loader:

        # Forward pass to get predictions
        y_pred = model(inputs)

        # Convert the predictions and targets to numpy arrays
        y_pred_np = y_pred.numpy()
        y_pred_all.append(y_pred_np)

# Concatenate the lists into a single array
y_pred_all = np.concatenate(y_pred_all)
df['DeepNN'] = y_pred_all
'''


/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- humid
- light
- temperature
Feature names seen at fit time, yet now missing:
- global_light_energy_w_m2
- humid_rh
- temperature_oc

  warnings.warn(message, FutureWarning)


"\ny_test_all = []\ny_pred_all = []\n\ndataset = MyDataset(x_train)\ndata_loader = DataLoader(dataset)\n\n# Disable gradient calculation\nwith torch.no_grad():\n    for inputs, label in data_loader:\n\n        # Forward pass to get predictions\n        y_pred = model(inputs)\n\n        # Convert the predictions and targets to numpy arrays\n        y_pred_np = y_pred.numpy()\n        y_pred_all.append(y_pred_np)\n\n# Concatenate the lists into a single array\ny_pred_all = np.concatenate(y_pred_all)\ndf['DeepNN'] = y_pred_all\n"

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

myDates = [datetime(2012,1,i+3) for i in range(10)]
myValues = [5,6,4,3,7,8,1,2,5,4]
fig, ax = plt.subplots()
bet = df.loc[df['datetime'].between('2023-06-09', '2023-06-13')]
ax.plot(bet['datetime'], bet['SVM'].astype(float), 'ro')

myFmt = DateFormatter("%m %d %H:%M")
ax.xaxis.set_major_formatter(myFmt)

## Rotate date labels automatically
fig.autofmt_xdate()
plt.show()

# Local Data

In [ ]:
bd_data = pd.read_csv("../datasets/GEMStat/India-Lake-2023-07-04_13-45/samples.csv", sep=';')

# Combine 'Date' and 'Time' columns into a single column
bd_data['datetime'] = pd.to_datetime(bd_data['Date'] + ' ' + bd_data['Time'], format='%Y-%m-%d %H:%M')

# Drop the original 'Date' and 'Time' columns if no longer needed
bd_data.drop(['Date', 'Time'], axis=1, inplace=True)

# Merge by datetime and LocationID
new_bd_data = pd.DataFrame()
new_bd_data[['LocationID','datetime', 'O2-Dis', ]] = bd_data[bd_data['Parameter'] == 'O2-Dis'][['LocationID', 'datetime','Value']]

print(bd_data.columns)

new_bd_data = pd.merge(new_bd_data, bd_data[bd_data['Parameter'] == 'pH'][['datetime', 'Value', 'LocationID']], on=['datetime', 'LocationID'], how='right')
new_bd_data = pd.merge(new_bd_data, bd_data[bd_data['Parameter'] == 'TEMP'][['datetime', 'Value', "LocationID"]], on=['datetime','LocationID'], how='right')

new_bd_data.rename(columns={'O2-Dis': 'dissolved_oxygen_mg_l', 'Value_x': 'ph', 'Value_y': 'temperature_oc'}, inplace=True)

print(len(new_bd_data))
new_bd_data.sample(10)

# Combine with Ammonia


In [38]:
from sklearn.ensemble import RandomForestClassifier

cis = pd.read_csv("../datasets/upsampled_data.csv")
test = cis.copy(True)
X_train,X_test,y_train,y_test = train_test_split(test[['DO_MGL','PH','precipitation (mm)','pressure_msl (hPa)', 'direct_radiation (W/m²)']], 
        test['decreaseFeed'], test_size=0.001)

# Training
clf = RandomForestClassifier(max_depth=40, criterion='gini', max_features='log2')
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=40, max_features='log2')

In [39]:
# Testing
'''
1. Remove outliers greater than 2 std devs on df
2. Impute missing values with median
3. Apply StandardScaler to DO, PH, pressure, direct radiation
'''
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

#imputer = SimpleImputer(strategy='median')
#df = imputer.fit_transform(df)
nh4_test = df.copy(True)
scaler = StandardScaler()
nh4_test[['RF', 'ph', 'pressure_msl (hPa)', 'direct_radiation (W/m²)']] = scaler.fit_transform(nh4_test[['RF', 'ph', 'pressure_msl (hPa)', 'direct_radiation (W/m²)']])

amm_pred = clf.predict(nh4_test[['RF', 'ph', 'precipitation (mm)', 'pressure_msl (hPa)', 'direct_radiation (W/m²)']])
df['ammonia'] = amm_pred



/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- RF
- ph
Feature names seen at fit time, yet now missing:
- DO_MGL
- PH

  warnings.warn(message, FutureWarning)


In [43]:
print(df.sample(10))
df.to_csv('../datasets/ammonia_preds.csv', index=False)


                     ph         temperature                   datetime  \
746                7.67               31.31 2023-06-18 22:26:23.707350   
522   7.987808965228319  31.578103749527713 2023-06-15 00:52:43.084418   
703  7.9073732718894005    31.9436144294844 2023-06-18 10:07:14.818315   
95    9.113908671973187   32.58740838275526 2023-06-09 10:23:24.981244   
508    8.02802681189778   32.12700789204939 2023-06-14 19:56:47.966372   
163   7.826937578550482   31.39597288319152 2023-06-10 07:27:11.153185   
8     8.532383745286966   33.83710149075512 2023-06-08 09:28:23.101344   
357   9.033472978634268     31.669324029471 2023-06-12 15:19:06.523164   
462    8.67151235860913  31.578103749527713 2023-06-14 04:01:19.195079   
531   7.947591118558858   31.48698698161479 2023-06-15 03:20:29.375885   

    do_linreg    rounded_datetime  airtemp  humid  precipitation (mm)  \
746     9.171 2023-06-18 22:00:00     28.4     93                 0.0   
522     9.476 2023-06-15 01:00:00     2